# Random forest model to predict free bici stands.
Using model that works best from predict_bicis.ipynb with some adjustments to features.

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [3]:
valenbisi = pd.read_csv("../data/VALENBISI.csv")
valenbisi

,Direction,Number,Active,Free_bici,Free_stand,Total_stands,Ticket,Update_date,Latitude,Longitude,Folder_datetime
0,"Colón, 60",16,1,1,19,20,1,2024-01-19 00:09:53,39.470092,-0.370433,2024-01-19 00:15:01
1,Plaza de Tetuán,9,1,0,25,25,1,2024-01-19 00:09:53,39.474355,-0.369930,2024-01-19 00:15:01
2,Micer Mascó - Rodriguez Fornos,81,1,5,15,20,0,2024-01-19 00:09:53,39.475128,-0.360978,2024-01-19 00:15:01
3,General Elio - Llano del Real,83,1,0,25,25,1,2024-01-19 00:09:53,39.477585,-0.366970,2024-01-19 00:15:01
4,Blasco Ibañez - Mestre Ripoll,100,1,18,2,20,0,2024-01-19 00:09:53,39.471634,-0.338150,2024-01-19 00:15:01
...,...,...,...,...,...,...,...,...,...,...,...
11545222,Alcasser - Poeta Alberto Lista,265,1,5,10,15,0,2025-04-14 23:49:17,39.470973,-0.408117,2025-04-15 00:00:06
11545223,Ninot - Regino Mas,270,1,0,16,16,0,2025-04-14 23:49:17,39.500075,-0.392889,2025-04-15 00:00:06
11545224,San Francisco de Paula - Castell de Pop,274,1,13,2,15,0,2025-04-14 23:49:17,39.448070,-0.333188,2025-04-15 00:00:06
11545225,Valle de la Ballestera - Pio Baroja,244,1,9,11,20,0,2025-04-14 23:49:17,39.478506,-0.406136,2025-04-15 00:00:06


In [4]:
# possible features to add: ######################################
# typical variations for each station (does it normally vary a lot or little)
# last number of free bikes
# last time updated

def feature_engineering(data):
    df = data.copy()
    # TIME
    # create hour of day as a sin/cosine feature
    df['hour_sin'] = np.sin(2 * np.pi * pd.to_datetime(df['Update_date']).dt.hour / 24)
    # season
    df['season'] = pd.to_datetime(df['Update_date']).dt.month % 12 // 3 + 1
    # weekday
    df['is_weekday'] = pd.to_datetime(df['Update_date']).dt.weekday < 5
    df['is_weekday'] = df['is_weekday'].astype(int)

    # since timedependent - add Free_stand from earlier timestep and time since this measurement
    # Sort by station and time 
    df['Update_date'] = pd.to_datetime(df['Update_date'])
    df = df.sort_values(by=['Number', 'Update_date'])
    # Create lag features grouped by station
    df['Prev_Free_stand'] = df.groupby('Number')['Free_stand'].shift(1)
    df['Prev2_Free_stand'] = df.groupby('Number')['Free_stand'].shift(2)
    df['Prev3_Free_stand'] = df.groupby('Number')['Free_stand'].shift(3)

    # Calculate time differences (in minutes)
    df['time_diff'] = df.groupby('Number')['Update_date'].diff().dt.total_seconds() / 60
    df['time_diff_2'] = df.groupby('Number')['Update_date'].diff(2).dt.total_seconds() / 60
    df['time_diff_3'] = df.groupby('Number')['Update_date'].diff(3).dt.total_seconds() / 60

    # Fill nan
    df[['Prev_Free_stand', 'Prev2_Free_stand', 'Prev3_Free_stand']] = df[
        ['Prev_Free_stand', 'Prev2_Free_stand', 'Prev3_Free_stand']
    ].fillna(method='bfill')

    df[['time_diff', 'time_diff_2', 'time_diff_3']] = df[
        ['time_diff', 'time_diff_2', 'time_diff_3']
    ].fillna(0)

    df = df.drop(columns=['Update_date', 'Folder_datetime'])

    # dont know free bici
    df = df.drop(columns=['Free_bici'])

    # Direction is unique in feature Number
    df = df.drop(columns=['Direction'])

    return df

In [5]:
prep_valenbisi = feature_engineering(valenbisi)
prep_valenbisi 

/var/folders/vs/5hqqq9px6_z7w4dnv2j4238m0000gn/T/ipykernel_47303/2264518456.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ].fillna(method='bfill')


,Number,Active,Free_stand,Total_stands,Ticket,Latitude,Longitude,hour_sin,season,is_weekday,Prev_Free_stand,Prev2_Free_stand,Prev3_Free_stand,time_diff,time_diff_2,time_diff_3
4376453,1,1,22,25,1,39.480042,-0.382929,0.000000,1,1,22.0,22.0,22.0,0.000000,0.000000,0.000000
4376718,1,1,22,25,1,39.480042,-0.382929,0.000000,1,1,22.0,22.0,22.0,11.183333,0.000000,0.000000
4377238,1,1,22,25,1,39.480042,-0.382929,0.000000,1,1,22.0,22.0,22.0,18.650000,29.833333,0.000000
4377459,1,1,23,25,1,39.480042,-0.382929,0.000000,1,1,22.0,22.0,22.0,10.133333,28.783333,39.966667
4377544,1,1,20,25,1,39.480042,-0.382929,0.258819,1,1,23.0,22.0,22.0,21.066667,31.200000,49.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8380731,299,1,0,2,1,40.448814,-3.584892,0.258819,3,0,0.0,0.0,0.0,0.000000,0.000000,0.000000
8381006,299,1,0,2,1,40.448814,-3.584892,0.258819,3,0,0.0,0.0,0.0,0.000000,0.000000,0.000000
8381227,299,1,0,2,1,40.448814,-3.584892,0.258819,3,0,0.0,0.0,0.0,0.000000,0.000000,0.000000
8381576,299,1,0,2,1,40.448814,-3.584892,0.258819,3,0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [6]:
import joblib

def save_model(model, X, filename):
    """Save the model to a file."""
    # .pkl
    joblib.dump((model, X.columns.tolist()), filename)
    #model, feature_names = joblib.load("model.pkl")

In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Define features and target
X = prep_valenbisi.drop(columns=['Free_stand'])
y = prep_valenbisi['Free_stand']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print performance metrics
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.2f}")

# Cross-validation
# cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')  # Using negative MSE for scoring
# print(f"Cross-validated MSE: {-cv_scores.mean():.2f}")

# Save the model
def save_model(model, X, filename):
    joblib.dump(model, filename)

save_model(model, X, "rf_model.pkl")



Mean Squared Error (MSE): 0.92
Mean Absolute Error (MAE): 0.42
R² Score: 0.98


In [8]:
# # Save the model
# def save_model(model, X, filename):
#     joblib.dump(model, filename)

# save_model(model, X, "rf_model.pkl")
